In [18]:
# !pip install httpx
import matplotlib
matplotlib.rcParams['font.sans-serif'] = "KaiTi"

## 网易接口

网易接口获取历史每日数据

URL 格式：


```
http://quotes.money.163.com/service/chddata.html?code=代码&start=开始时间&end=结束时间&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP
```


参数说明：代码为股票代码，上海股票前加0，如600756变成0600756，深圳股票前加1。时间都是6位标识法，年月日，fields标识想要请求的数据。可以不变。

例如大盘指数数据查询（上证指数000001前加0，沪深300指数000300股票前加0，深证成指399001前加1，中小板指399005前加1，创业板指399006前加1）：

```
http://quotes.money.163.com/service/chddata.html?code=0000300&start=20151219&end=20171108&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER
```


In [26]:
import httpx
import pandas as pd
import dtale
from io import StringIO 

url = "http://quotes.money.163.com/service/chddata.html?code=0000001&start=20220101&end=20220723&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP"
r = httpx.get(url)
r.encoding = 'gb18030'
data = pd.read_csv(StringIO(r.text), index_col=0, parse_dates=[0])
dtale.show(data)

## Souhu 数据接口

搜狐数据接口：

http://q.stock.sohu.com/hisHq?code={code}&start={start}&end={end}，{code}替换为股票代码，大陆股票代码加前缀cn_。{start}替换为起始日期，{end}替换为结束日期。


例如：在浏览器地址栏输入：

http://q.stock.sohu.com/hisHq?code=cn_601766,cn_000002&start=20150601&end=20160414

数组中数据一次对应含义如下：

日期,
今日开盘价,
今日收盘价,
涨跌幅度，
涨跌百分比,
今日最低价,
今日最高价，
今日成交股票手数,
今日成交金额(单位：万)，
换手率

In [25]:
import json
url = "https://q.stock.sohu.com/hisHq?code=cn_600036&start=20220101&end=20220723"
r = httpx.get(url)
data = json.loads(r.text)
stock = data[0]['hq']
df = pd.read_json(json.dumps(stock), orient="records")
df.columns = ['日期', '开盘价','收盘价','涨跌幅度', '涨跌百分比','最低价', '最高价', '成交股票手数', '成交金额(单位：万)', '换手率']

df = df.set_index("日期")
df.index = pd.to_datetime(df.index)
dtale.show(df)

## 新浪财经基金净值数据

url：

http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20220101&dateto=20220723&page=1

接口是分页的形式，需要遍历 page。

```
'日期' => 'fbrq',
'单位净值（元）' => 'jjjz',
'累计净值（元）' => 'ljjz',
```

In [28]:
page = 1
total = pd.DataFrame([])

while True:
    url = f'http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page={page}'
    print(f'fetch url: {url}')
    r = httpx.get(url)
    data = json.loads(r.text)
    result = data["result"]["data"]['data']
    if (len(result) == 0):
        break
    df = pd.read_json(json.dumps(result), orient="records")
    total = pd.concat([total, df])
    page = page + 1


total.columns = ['日期', '单位净值（元）', '累计净值（元）']
total = total.set_index('日期')
total.index = pd.to_datetime(total.index)
dtale.show(total)

fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=1
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=2
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=3
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=4
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=5
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInfoService.getNav?symbol=040011&datefrom=20210101&dateto=20220723&page=6
fetch url: http://stock.finance.sina.com.cn/fundInfo/api/openapi.php/CaihuiFundInf